In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn import metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
import glob

In [2]:

print("Total number of subjects:", len(all_data_files))
# print("Total number of subjects:", len(all_generated_data_files))
# print(all_generated_data_files)

NameError: name 'all_data_files' is not defined

In [5]:
class KeystrokesDataset(Dataset):
    def __init__(self, samples, labels, transform):
        self.samples = samples
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = self.samples[idx]
        x = x.transpose((1, 2 ,0))
        x = self.transform(image=x)['image']
        label = torch.from_numpy(self.labels[idx]).float()
        return x, label


In [6]:
class KeystrokeImageNetwork(nn.Module):
  
    def __init__(self):
        super(KeystrokeImageNetwork, self).__init__()

        # 10 fingers
        # => output (x, 10)
        self.conv1_1 = nn.Conv2d(5, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        # self.conv3_1 = nn.Conv2d(5, 64, kernel_size=(1,42), padding=0)
        # self.conv4_1 = nn.Conv2d(5, 64, kernel_size=(43,1), padding=0)

        # self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        # self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        # self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        # max pooling (kernel_size, stride)
        # self.pool = nn.AvgPool2d(2, 2)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        # self.fc6 = nn.Linear(42*42*5, 512)
        self.fc6 = nn.Linear(10*10*128, 512)
        # self.fc6 = nn.Linear(10*10*128 + 64*43 + 64*42, 512)
        self.fc7 = nn.Linear(512, 64)
        self.fc8 = nn.Linear(64, 1)

    def forward(self, x, training=True):
        # beforeX = F.relu(self.conv3_1(x))
        # afterX = F.relu(self.conv4_1(x))

        # print("beofore x shape is %s, after x shape is %s"%(beforeX.shape, afterX.shape)) #64 x 64 x 43 x 1 #64 x 64 x 1 x 42

        # probelm of the sparse feature
        # need to be fixed in the feature engineering
        # Add two new channel 
        # beforeX = beforeX.view(-1, 64 * 43 * 1)  # represent bigraph (*) - (correspending key)
        # afterX = afterX.view(-1, 64 * 1 * 42) # represent bigraph (corresponding key) - (*)

        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)

        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)

        # x = F.relu(self.conv3_1(x))
        # x = F.relu(self.conv3_2(x))
        # x = F.relu(self.conv3_3(x))
        # x = self.pool(x)

        # x = x.view(-1, 42*42*5)
        x = x.view(-1, 10*10*128)

        # x = torch.cat((x, beforeX, afterX), 1)
        # x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        # x = F.dropout(x, 0.5, training=training)
        x = self.fc8(x)

        # x = F.log_softmax(x, dim=1)
        x = torch.sigmoid(x)
        return x  

In [7]:

#EXTRACTS REAL AND GENERATED SAMPLES
def extractRealAndGeneratedData(user_num, all_data_files,all_generated_data_files, positive_only = False, gan_only = False):
    positive_data = None
    negative_data = None

    positive_init = False
    negative_init = False

    positive_indices = None
    negative_indices = None

    positive_data = list()
    negative_data = list()
    target = user_num
    total = 40000
    
    #print('**********************', target)
    
    if(positive_only):
        for i, filename in enumerate(all_data_files):
            if i == target:
                positive_dataset = np.load(filename)
                positive_indices = list(range(len(positive_dataset)))
                np.random.shuffle(positive_indices)
                positive_init = True
                positive_data = positive_dataset[positive_indices]
                #print("Current positive keystroke images Data shape is",positive_data.shape)
        return positive_data
    
    if(gan_only):
        for i, filename in enumerate(all_generated_data_files):
            if i == target:
                negative_dataset = np.load(filename)
                negative_indices = list(range(len(negative_dataset)))
                np.random.shuffle(negative_indices)
                negative_init = True
                negative_data = negative_dataset[negative_indices]
                #print("Current negative keystroke images Data shape is",negative_data.shape)
        return negative_data
        
    else:
        for i, filename in enumerate(all_data_files):
            if i == target:
                positive_dataset = np.load(filename)
                # positive_indices = list(range(len(positive_dataset)))
                # np.random.shuffle(positive_indices)
                # positive_init = True
                # positive_data = positive_dataset[positive_indices]
                #print("Current positive keystroke images Data shape is",positive_data.shape)
    
        for i, filename in enumerate(all_generated_data_files):
            if i == target:
                negative_dataset = np.load(filename)
                # negative_indices = list(range(len(negative_dataset)))
                # np.random.shuffle(negative_indices)
                # negative_init = True
                # negative_data = negative_dataset[negative_indices]
                #print("Current negative keystroke images Data shape is",negative_data.shape)
        return positive_dataset, negative_dataset


In [8]:
def getNegative(target,all_data_files):
    positive_length = 100
    negative_length = 35000
    negative_data = None
    negative_init = False
        
    for i, filename in enumerate(all_data_files):
            if i != target:
                negative_dataset = np.load(filename)
                nega_len = len(negative_dataset)
                if random.randint(0, 1) == 0:
                    smaple_len = math.floor(nega_len/negative_length * positive_length)
                else:
                    smaple_len = math.ceil(nega_len/negative_length * positive_length)
                negative_indices = list(range(nega_len))

                if not negative_init:
                    negative_data = negative_dataset[negative_indices[:smaple_len]]
                    negative_init = True
                else:
                    extend_sameple = negative_dataset[negative_indices[:smaple_len]]
                    negative_data = np.concatenate((negative_data, extend_sameple), axis=0)
    #print("Negative Data Shape:",negative_data.shape)
    return negative_data


In [13]:
## EVALUATING GENERATED DATA  
model = KeystrokeImageNetwork()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
knum = 0
# list of test results to save and average at the end (CNN [50,75,100] vs [WGAN,DCGAN,CGAN][50,75,100]) 27 different lists
cnn_test_data = {'WGAN':[],'DCGAN':[],'CGAN':[]}
cnn_pos_data = list() # cnn tested on only positive ("1) data from dataset
cnn_gan_data = list() # cnn tested on gan data ("1") labeled real
cnn_pos_and_gan_data = list()# cnn tested on positive and real data labeled 1
cnn_pos_vs_gan_data = list() # cnn tested on positive data ("1") vs generated data labeled ("0")
cnn_pos_vs_neg_data = list() # 
cnn_gan_vs_neg_data = list() # 
cnn_pos_and_gan_vs_neg_data = list() #

def testCNN():
    for gan in ['WGAN','DCGAN']:
        
        
        for keystroke_num in ['50']:
            print("finish")
            cnn_pos_data = list() # cnn tested on only positive ("1) data from dataset
            cnn_gan_data = list() # cnn tested on gan data ("1") labeled real
            cnn_pos_and_gan_data = list()# cnn tested on positive and real data labeled 1
            cnn_pos_vs_gan_data = list() # cnn tested on positive data ("1") vs generated data labeled ("0")
            cnn_pos_vs_neg_data = list() # 
            cnn_gan_vs_neg_data = list() # 
            cnn_pos_and_gan_vs_neg_data = list() #
            path = r"/home/jupyter/src/Thesis_Project/Data/"+keystroke_num
            all_data_files = glob.glob(path + "/*")
            all_data_files.sort()
            path = r"/home/jupyter/src/Thesis_Project/"+gan+"_data/"+keystroke_num
            all_generated_data_files = glob.glob(path + "/*")
            all_generated_data_files.sort()
            path = r"/home/jupyter/src/Thesis_Project/CNN_" +keystroke_num+"_models"
            all_cnns = glob.glob(path + "/*")
            all_cnns.sort()
           
            for k in range(75):
                # POS + GAN (i.e. both are labeled ones)
                positive_data, gan_data = extractRealAndGeneratedData(k,all_data_files,all_generated_data_files)
                total_dataset = np.concatenate((positive_data, gan_data), axis = 0)
                total_labels = np.concatenate((
                  np.ones((len(positive_data), 1)), 
                  np.ones((len(gan_data), 1))
                ), axis=0)
                test_transform = getTransform(total_dataset)
                test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
                test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

                #Loading and testing model 
                model.load_state_dict(torch.load(all_cnns[k]))
                model.to(device)
                model.eval()

                # Test each CNN against Generated Data
                test_acc = evaluate(model,test_dataloader)


                # Summarize results 

                # fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=1)
                # fnr = 1 - tpr
                # eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
                # eer = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
                # print(fpr,tpr,fnr)
                # print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
                cnn_pos_and_gan_data.append(test_acc)


                # POS ONLY
                total_dataset = positive_data
                total_labels = np.ones((len(positive_data), 1))
                test_transform = getTransform(total_dataset)
                test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
                test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
                test_acc = evaluate(model,test_dataloader)
                # print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
                cnn_pos_data.append(test_acc)


                # GAN ONLY
                total_dataset = gan_data
                total_labels = np.ones((len(gan_data), 1))
                test_transform = getTransform(total_dataset)
                test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
                test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
                test_acc = evaluate(model,test_dataloader)
                # print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
                cnn_gan_data.append(test_acc)


                # POS vs GAN ONLY
                total_dataset = np.concatenate((positive_data, gan_data), axis = 0)
                total_labels = np.concatenate((
                  np.ones((len(positive_data), 1)), 
                  np.zeros((len(gan_data), 1))
                ), axis=0)
                test_transform = getTransform(total_dataset)
                test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
                test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
                test_acc = evaluate(model,test_dataloader)
                # print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
                cnn_pos_vs_gan_data.append(test_acc)
                
                # GRAB  NEGATIVE DATA
                negative_data = getNegative(k,all_data_files)
    
                # POS vs NEG 
                total_dataset = np.concatenate((positive_data, negative_data), axis = 0)
                total_labels = np.concatenate((
                  np.ones((len(positive_data), 1)), 
                  np.zeros((len(negative_data), 1))
                ), axis=0)
                test_transform = getTransform(total_dataset)
                test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
                test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
                test_acc = evaluate(model,test_dataloader)
                # print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
                cnn_pos_vs_neg_data.append(test_acc)

                
                # GAN vs NEG
                total_dataset = np.concatenate((gan_data, negative_data), axis = 0)
                total_labels = np.concatenate((
                  np.ones((len(gan_data), 1)), 
                  np.zeros((len(negative_data), 1))
                ), axis=0)
                test_transform = getTransform(total_dataset)
                test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
                test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
                test_acc = evaluate(model,test_dataloader)
                # print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
                cnn_gan_vs_neg_data.append(test_acc)

                
                # POS + GAN vs NEG 
                total_dataset = np.concatenate((positive_data,np.concatenate((gan_data, negative_data), axis = 0)),axis=0)
                total_labels = np.concatenate((
                  np.ones((len(positive_data)+len(gan_data), 1)), 
                  np.zeros((len(negative_data), 1))
                ), axis=0)
                test_transform = getTransform(total_dataset)
                test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
                test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
                test_acc = evaluate(model,test_dataloader)
                # print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
                cnn_pos_and_gan_vs_neg_data.append(test_acc)
                
                
    
            cnn_test_data[gan].append([cnn_pos_data,cnn_gan_data,cnn_pos_and_gan_data,cnn_pos_vs_gan_data,cnn_pos_vs_neg_data,cnn_gan_vs_neg_data, cnn_pos_and_gan_vs_neg_data])
        

testCNN()


finish
finish


In [14]:
print("Here")
test_name = ["cnn_pos_data\t\t\t","cnn_gan_data\t\t\t","cnn_pos_and_gan_data\t\t","cnn_pos_vs_gan_data\t\t","cnn_pos_vs_neg_data\t\t","cnn_gan_vs_neg_data\t\t", "cnn_pos_and_gan_vs_neg_data\t"]
knum = ['50','50','100']
print("~~~~~~~~~RESULTS~~~~~~~~~")
for g in ['WGAN','DCGAN']:
    k = 0
    for i in cnn_test_data[g]:
        t = 0
        for tests in i:
            print(g +'\t',knum[k]+" keystrokes\t",test_name[t] + "AVERAGE:",round(sum(tests)/len(tests),3))
            t += 1
        k += 1
        print("\n\n")
    

Here
~~~~~~~~~RESULTS~~~~~~~~~
WGAN	 50 keystrokes	 cnn_pos_data			AVERAGE: 96.888
WGAN	 50 keystrokes	 cnn_gan_data			AVERAGE: 88.688
WGAN	 50 keystrokes	 cnn_pos_and_gan_data		AVERAGE: 97.376
WGAN	 50 keystrokes	 cnn_pos_vs_gan_data		AVERAGE: 60.176
WGAN	 50 keystrokes	 cnn_pos_vs_neg_data		AVERAGE: 93.532
WGAN	 50 keystrokes	 cnn_gan_vs_neg_data		AVERAGE: 92.125
WGAN	 50 keystrokes	 cnn_pos_and_gan_vs_neg_data	AVERAGE: 95.353



DCGAN	 50 keystrokes	 cnn_pos_data			AVERAGE: 96.888
DCGAN	 50 keystrokes	 cnn_gan_data			AVERAGE: 82.688
DCGAN	 50 keystrokes	 cnn_pos_and_gan_data		AVERAGE: 97.383
DCGAN	 50 keystrokes	 cnn_pos_vs_gan_data		AVERAGE: 60.183
DCGAN	 50 keystrokes	 cnn_pos_vs_neg_data		AVERAGE: 93.646
DCGAN	 50 keystrokes	 cnn_gan_vs_neg_data		AVERAGE: 92.135
DCGAN	 50 keystrokes	 cnn_pos_and_gan_vs_neg_data	AVERAGE: 95.22





#### positive_indices = list(range(3))
positive = [[1,2,3],[1,1,1],[2,2,2]]
np.random.shuffle(positive_indices)
positive_data = positive[positive_indices]

In [10]:

import os 
cnn_pos_data = list() # cnn tested on only positive ("1) data from dataset
cnn_gan_data = list() # cnn tested on gan data ("1") labeled real
cnn_pos_and_gan_data = list()# cnn tested on positive and real data labeled 1
cnn_pos_vs_gan_data = list() # cnn tested on positive data ("1") vs generated data labeled ("0")
cnn_pos_vs_neg_data = list() # 
cnn_gan_vs_neg_data = list() # 
cnn_pos_and_gan_vs_neg_data = list() #


    
    
path = r"/home/jupyter/src/Thesis_Project/Data/100"
all_data_files = glob.glob(path + "/*")
all_data_files.sort()
path = r"/home/jupyter/src/Thesis_Project/WGAN_data/100"
all_generated_data_files = glob.glob(path + "/*")
all_generated_data_files.sort()
path = r"/home/jupyter/src/Thesis_Project/CNN_100_models"
all_cnns = glob.glob(path + "/*")
all_cnns.sort()
# print(all_generated_data_files)
    

#Loading and testing model 
model.load_state_dict(torch.load(all_cnns[0]))
model.to(device)
model.eval()
l=0
for k in range(75):
    # POS + GAN (i.e. both are labeled ones)
    positive_data, g = extractRealAndGeneratedData(k,all_data_files,all_generated_data_files)
    p, gan_data = extractRealAndGeneratedData(k,all_data_files,all_generated_data_files)
    # are_same = np.array_equal(positive_data,p)
    # are_same1 = np.array_equal(gan_data,g)
    # if are_same:
    #     print("The arrays are the same.")
    # else:
    #     print("The arrays are different.")
    # if are_same1:
    #     print("The arrays are the same.")
    # else:
    #     print("The arrays are different.")  

    total_dataset = gan_data
    total_labels = np.ones((len(gan_data), 1))
    # total_dataset = np.concatenate((positive_data, gan_data), axis = 0)
    # total_labels = np.concatenate((
    #   np.ones((len(positive_data), 1)), 
    #   np.ones((len(gan_data), 1))
    # ), axis=0)
    test_transform = getTransform(total_dataset)
    test_dataset = KeystrokesDataset(total_dataset, total_labels, test_transform)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

    #Loading and testing model 
    model.load_state_dict(torch.load(all_cnns[k]))
    model.to(device)
    model.eval()
    test_acc = evaluate(model,test_dataloader)
    l += test_acc


    print('Accuracy of the network on the test inputs: %f %%' % (test_acc))
print('Accuracy of the network on the test inputs: %f %%' % (l/48))


NameError: name 'model' is not defined

In [17]:
import os
directory = '/home/jupyter/src/Thesis_Project/WGAN_data/50/'
prefix = 'gen_data_'
extension = '.npy'

#Get the list of files to be renamed
files = os.listdir(directory)
files = [f for f in files if f.startswith(prefix) and f.endswith(extension)]
files = sorted(files, key=lambda x: int(x.split('.')[0].split('_')[-1]))
print(files)

#Rename the files
for i, f in enumerate(files):
    old_name = os.path.join(directory, f)
    new_name = os.path.join(directory, prefix + str(i).zfill(3) + extension)
    os.rename(old_name, new_name)

['gen_data_0.npy', 'gen_data_000.npy', 'gen_data_1.npy', 'gen_data_001.npy', 'gen_data_2.npy', 'gen_data_002.npy', 'gen_data_3.npy', 'gen_data_003.npy', 'gen_data_4.npy', 'gen_data_004.npy', 'gen_data_5.npy', 'gen_data_005.npy', 'gen_data_6.npy', 'gen_data_006.npy', 'gen_data_007.npy', 'gen_data_7.npy', 'gen_data_8.npy', 'gen_data_008.npy', 'gen_data_009.npy', 'gen_data_9.npy', 'gen_data_10.npy', 'gen_data_010.npy', 'gen_data_11.npy', 'gen_data_011.npy', 'gen_data_012.npy', 'gen_data_12.npy', 'gen_data_13.npy', 'gen_data_013.npy', 'gen_data_14.npy', 'gen_data_014.npy', 'gen_data_15.npy', 'gen_data_015.npy', 'gen_data_016.npy', 'gen_data_16.npy', 'gen_data_17.npy', 'gen_data_017.npy', 'gen_data_18.npy', 'gen_data_018.npy', 'gen_data_019.npy', 'gen_data_19.npy', 'gen_data_20.npy', 'gen_data_020.npy', 'gen_data_21.npy', 'gen_data_021.npy', 'gen_data_022.npy', 'gen_data_22.npy', 'gen_data_23.npy', 'gen_data_023.npy', 'gen_data_024.npy', 'gen_data_24.npy', 'gen_data_025.npy', 'gen_data_25.n

In [9]:
def evaluate(model,test_dataloader):
    correct = 0.0
    total = 0.0

    scores = []
    y = []

    with torch.no_grad():
        for i, data in enumerate(test_dataloader):
            inputs, labels = data
            #images = images.to(device).half() # uncomment for half precision model
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            #Passes input through CNN and generates output
            outputs = model.forward(inputs, training=False)
            total += labels.size(0)

            scores.extend(outputs.cpu().numpy().reshape(len(outputs)))
            y.extend(labels.cpu().numpy().reshape(len(labels)))
            
            #scores are then aggregated and averaged
            predicted = (outputs > 0.3).float()
            correct += (predicted == labels).sum().item()

    test_acc = (100.0 * correct) / total
    return test_acc

def getTransform(total_dataset):
    mean = 0.
    std = 0.

    # batch size (the last batch can have smaller size!)
    batch_samples = len(total_dataset) 
    images = total_dataset.reshape(batch_samples, 5, -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)

    mean /= len(total_dataset)
    std /= len(total_dataset)


    test_transform = A.Compose([
    A.Normalize(mean=mean.tolist(), std=std.tolist(), max_pixel_value=1.0, p=1.0),
    ToTensorV2(p=1.0),
    ], p=1)
    return test_transform